In [ ]:
"""Maintaining an instrument master

Attributes
----------
instruments
properties
search
"""

## Instruments

LUSID treats instruments as a first class object. They are at the core of much of LUSID's functionality and are required to post transactions and holdings. Read more about how instruments are treated inside LUSID here [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument).

In this notebook we will cover how to work with instruments inside LUSID. We will:

1) Locally import our instrument universe from a CSV file

2) Determine what identifiers we can use for our instruments

3) Upsert our instruments into LUSID

4) Test that we can retrieve an instrument by its identifier

5) Add some reference data to our instruments e.g. security type

6) Search for all equity instruments using the security type reference data we added

7) Update the identifier for one of our instruments

8) Delete a handful of instruments 

*First things first, run the cell below to import the libraries and create the LUSID client*

In [ ]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import uuid
import os

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

### 1) Import our instrument universe

The first thing that we need to do is get the information about our instruments. In this case we are going to import our instrument universe from a CSV file which has been exported from another one of our systems. 

*Run the cell below to import our instruments and show the first 15 securities*

In [ ]:
# Import our instruments from a CSV file using Pandas
instruments = pd.read_csv('data/instruments.csv')
# View the first 15 instruments
instruments.head(n=15)

Here we can see that our instrument universe contains a mix of equities and bonds. Each instrument has a number of identifiers attached to it as well as some basic reference information. 

### 2) Get allowable instrument identifiers

Great now we have our instrument information we want to add our instruments to LUSID.

To add each instrument to LUSID we must have at least one unique identifier for it. 

This is because LUSID uses a bi-temporal data store, which means that no instrument is ever truly deleted. There are thus restrictions on what can and can't be used as a unique identifier. This ensures that there are no collisions between instruments which would arise from using an identifier which may be re-purposed. Read more about LUSID's bi-temporal data store here [LUSID Knowledge Base: Bi-Temporal Data Store](https://support.lusid.com/what-is-bi-temporal-data).

We can call LUSID to find out what the allowable types of instrument indentifiers are. Read more about identifier schemes in LUSID here [LUSID Knowledge Base: Identifier Schemes](https://support.lusid.com/which-instrument-identifier-schemes-should-i-use-with-lusid). 

For further usage of the get instrument identifiers API call refer to the [LUSID API Docs: Get Allowable Instrument Identifiers](https://docs.lusid.com/#operation/GetInstrumentIdentifiers).

*Run the cell below to get our allowable instrument identifiers*

In [ ]:
# Call LUSID to get our allowable instrument indentifiers
response = api_factory.build(lusid.api.InstrumentsApi).get_instrument_identifier_types()

# Extract the unique identifiers from the response and print them
prettyprint.get_identifiers(response, unique=True)

These are the unique identifiers that we can use when adding our instruments to LUSID. We must provide at least one of them for each instrument. 

We can also find the available non-unique identifiers.

*Run the cell below to see the non-unique identifiers that are available*

In [ ]:
# Extract the non-unique identifiers from the response and print them
prettyprint.get_identifiers(response, unique=False)

### 3) Upsert instruments

Now that we know which instrument identifiers we can use, we can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument).

As part of this definition we will also attach two alias identifiers to our instruments. Read more about alias identifiers here [LUSID Knowledge Base: What is an Alias Identifier?](https://support.lusid.com/alias-identifiers).

We use an upsert method to add instrument definitions to the instrument universe. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert our instruments into LUSID*

In [ ]:
# Initialise dictionary to hold our instrument definitions
definitions = {}

# Iterate over each row in our instrument universe
for row, instrument in instruments.iterrows():
    
    # Specify your identifier columns
    identifier_columns = [
        ('isin', 'Isin'), 
        ('figi', 'Figi'), 
        ('ticker', 'Ticker'),
        ('client_internal', 'ClientInternal')
    ]
    
    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier[1]] = models.InstrumentIdValue(
            value=instrument[identifier[0]])
    
    # Create our definitions
    definitions[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers)
    
# Call the LUSID API to upsert our instrument definitions
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(request_body=definitions)

# Print each the definition of each instrument returned in the response
for figi, instrument in response.values.items():
    print('Figi: ' + figi + '\n',
          'Instrument Name: ' + instrument.name + '\n', 
          'LUSID Instrument ID: ' + instrument.lusid_instrument_id + '\n'+'\n')

We have added our instruments to LUSID! We can now use these instruments to book trades and set holdings.

As you can see every instrument that is upserted into LUSID is given a LUSID unique instrument Identifier. This is a globally unique identifier for the instrument and is used in some calls to LUSID. Read more about LUSID Unique Instrument Identifiers here [LUSID Knoweldge Base: LUSID Instrument Identifiers](https://support.lusid.com/what-is-a-lusid-unique-identifier-luid).

### 4) Retrieve an Instrument by its Identifier

In addition to listing all instruments we can also retrieve an instrument individually by searching for it by one of its unique identifiers. 

Here we can look up one of our instruments by its Financial Instrument Global Identifier (FIGI). 

For further usage of the get instrument API call refer to the [LUSID API Docs: Get Instrument Defintion](https://docs.lusid.com/#operation/GetInstrument).

*Run the cell below to retrieve our instrument's definition by its FIGI*

In [ ]:
# Call the LUSID API to retrieve an instrument by its FIGI
response = api_factory.build(lusid.api.InstrumentsApi).get_instrument(
    identifier_type='Figi',
    identifier='BBG000C05BD1')

# Print the response with our instrument definition
print('Instrument Name: ' + response.name + '\n', 
      'FIGI: ' + response.identifiers['Figi'] + '\n',
      'ClientInternal: ' + response.identifiers['ClientInternal'] + '\n',
      'LUSID Instrument ID: ' + response.lusid_instrument_id)

As we gave each of our instruments two unique identifiers, i.e. a FIGI and a ClientInternal, we can also look up the same instrument by its ClientInternal identifier.

*Run the cell below to retrieve our instrument definition by its ClientInternal identifier*

In [ ]:
# Call the LUSID API to retrieve an instrument by its ClientInternal ID
response = api_factory.build(lusid.api.InstrumentsApi).get_instrument(
    identifier_type='ClientInternal',
    identifier='imd_43535553')

# Print the response with our instrument
print('Instrument Name: ' + response.name + '\n', 
      'FIGI: ' + response.identifiers['Figi'] + '\n',
      'ClientInternal: ' + response.identifiers['ClientInternal'] + '\n',
      'LUSID Instrument ID: ' + response.lusid_instrument_id)

We can also retrieve multiple instruments by passing multiple values of a given identifier into the get instruments method. For further usage of the get instruments API call refer to the [LUSID API Docs: Get Instrument Defintion](https://docs.lusid.com/#operation/GetInstruments).

*Run the cell below to retrieve multiple instrument definitions by their FIGI*

In [ ]:
# Call the LUSID API to retrieve all instruments with a Figi in the provided list
response = api_factory.build(lusid.api.InstrumentsApi).get_instruments(
    identifier_type='Figi',
    request_body=['BBG000C05BD1', 'BBG000DQQNJ8'])

# Print the response with our instrument defintions
for figi, instrument in response.values.items():
    print('Instrument Name: ' + instrument.name + '\n', 
          'FIGI: ' + figi + '\n',
          'ClientInternal: ' + instrument.identifiers['ClientInternal'] + '\n',
          'LUSID Instrument ID: ' + instrument.lusid_instrument_id + '\n')

### 5) Upsert instrument properties

In addition to our unique identifier and aliases we can also add properties to our instruments. 

To define a property we need to create a property definition and then call LUSID to create that property. Read more about property defintions here [LUSID Knowledge Base: Property Definitions](https://support.lusid.com/what-is-a-property-definition). 

Note that every property is defined by a data type which is referenced by a data type id. Read more about data types here [LUSID Knowledge Base: Data Types](https://support.lusid.com/what-is-a-data-type).

Let us create a property to hold the asset class of our instrument.

For further usage of the create property definition API call refer to the [LUSID API Docs: Create Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition).

*Run the cell below to create our security type property*

In [ ]:
# Define the scope that we will use for our properties
properties_scope = 'custom_properties_{}'.format(str(uuid.uuid4())[:4])

# Create the request for our new property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Instrument',
    scope=properties_scope,
    code='asset_class',
    value_required=True,
    display_name='asset_class',
    life_time='TimeVariant',
    data_type_id=models.ResourceId(
        scope='system', 
        code='string'))

# Call LUSID to create our new property
property_response = api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
    create_property_definition_request=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
asset_class_property_key = property_response.key

# Print our property key
print (asset_class_property_key)

As you can see from the call above every property is identified by its key. Read more about properties and their keys here [LUSID Knowledge Base: Properties](https://support.lusid.com/what-is-a-property-definition).

We can now add this property to each of our instruments.

For further usage of the upsert instrument properties API call refer to the [LUSID API Docs: Upserting Instrument Properties](https://docs.lusid.com/#operation/UpsertInstruments)

*Run the cell below to set our asset class property for each of our instruments*

Now that we have created our property we can set it across our instrument universe.

*Run the cell below to set our asset class property for each of our instruments*

In [ ]:
requests = []

for row in instruments.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property = models.ModelProperty(
        key=asset_class_property_key,
        value=models.PropertyValue(
            label_value=instrument['market_sector'])
    )    
    
    # Build our request to set our property
    requests.append(
        models.UpsertInstrumentPropertyRequest(
            identifier_type='Figi',
            identifier=instrument['figi'],
            properties=[asset_class_property]))

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(
    upsert_instrument_property_request=requests)

# Print our response

print ('Instrument properties upserted at {}'.format(
    response.as_at_date))

We can now call LUSID to retrieve an instrument definition for one of our instruments with our user defined security type property attached. We do this by speicfying the property key in the instrument_property_keys argument of the method. 

*Run the cell below to call LUSID and retrieve our instrument*

In [ ]:
# Call LUSID to get the ISIN and security type for one of our instruments
response = api_factory.build(lusid.api.InstrumentsApi).get_instrument(
    identifier_type='Figi',
    identifier='BBG000C05BD1',
    property_keys=[
        asset_class_property_key
    ])

# Print the response
print('Instrument Name: ' + response.name + '\n', 
      'Figi: ' + response.identifiers['Figi'] + '\n',
      'ClientInternal: ' + response.identifiers['ClientInternal'] + '\n',
      'Isin: ' + response.identifiers['Isin'] + '\n',
      'Ticker: ' + response.identifiers['Ticker'] + '\n',
      'Lusid Instrument ID: ' + response.lusid_instrument_id + '\n',
      'Asset Class: ' + response.properties[0].value.label_value + '\n')

We can also delete a property using the same upsert properties endpoint. We do this by specifying the property key that we'd like to delete. 

*Run the cell below to delete the asset class property for one of our instruments*

In [ ]:

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).delete_instrument_properties(
    identifier_type='Figi',
    identifier="BBG000C05BD1",
    request_body=[
          asset_class_property_key
    ],
    effective_at=datetime.now(pytz.UTC)
)

# Print our response
print ('Properties deleted at {}'.format(
    response.as_at))

### 6) Search instruments

Now that we have set some properties on our instruments, we can now go ahead and search our instrument universe that we've just created using these properties.

In this case let's search for all equity instruments using our asset class property.

For further usage of the instruments search API call refer to the [LUSID API Docs: Search Instruments](https://docs.lusid.com/#operation/InstrumentsSearch)

*Run the cell below to search for all our equity securities*

In [ ]:
# Build our serach request
search_request_internal = models.InstrumentSearchProperty(
    key=asset_class_property_key,
    value='equity')

# Call LUSID to search for instruments
response = api_factory.build(lusid.api.SearchApi).instruments_search(
    instrument_search_property=[search_request_internal],
    mastered_only=True)

# Print the instrument definitions from our response
for instrument in response[0].mastered_instruments:
    print('Instrument Name: ' + instrument.name + '\n', 
          'Figi: ' + instrument.identifiers['Figi'].value + '\n',
          'ClientInternal: ' + instrument.identifiers['ClientInternal'].value + '\n',
          'LUSID Instrument ID: ' + instrument.identifiers['LusidInstrumentId'].value +'\n')

Using search we can also search for instruments that are not currently in the instrument universe inside LUSID. 

*Run the cell below to search for an instrument that is not in our instrument universe*

In [ ]:
search_request_external = models.InstrumentSearchProperty(
    key='Instrument/default/Figi',
    value='BBG000BD5TW4')

response = api_factory.build(lusid.api.SearchApi).instruments_search(
    instrument_search_property=[search_request_external],
    mastered_only=False)

for instrument in response[0].external_instruments:
    print ('Instrument Name: ' + instrument.name + '\n', 
          'Figi: ' + instrument.identifiers['Figi'].value)
    for _property in instrument.properties:
        print (_property.key, '-', _property.value.label_value)
    print ('\n')

We can also chain our search requests together. When we use multiple search requests LUSID uses OR logic.

*Run the cell below to use both our search requests together*

In [ ]:
response = api_factory.build(lusid.api.SearchApi).instruments_search(
    instrument_search_property=[search_request_internal, search_request_external],
    mastered_only=False)

for instrument in response[0].mastered_instruments:
    print('Instrument Name: ' + instrument.name + '\n', 
          'Figi: ' + instrument.identifiers['Figi'].value + '\n',
          'ClientInternal: ' + instrument.identifiers['ClientInternal'].value + '\n',
          'LUSID Instrument ID: ' + instrument.identifiers['LusidInstrumentId'].value +'\n')

for instrument in response[1].external_instruments:
    print ('Instrument Name: ' + instrument.name + '\n', 
          'Figi: ' + instrument.identifiers['Figi'].value)
    for _property in instrument.properties:
        print (_property.key, '-', _property.value.label_value)
    print ('\n')

### 7) Update instrument identifier

We can also update an instruments identifiers. For further usage of the update instrument identifier API call refer to the [LUSID API Docs: Update an Instrument Identifier](https://docs.lusid.com/#operation/UpdateInstrumentIdentifier).

*Run the cell below to update the Client Internal Id for one of our instruments*

In [ ]:
request = models.UpdateInstrumentIdentifierRequest(
    type='ClientInternal',
    value='imd_43535554',
    effective_at=None)

response = api_factory.build(lusid.api.InstrumentsApi).update_instrument_identifier(
    identifier_type='Figi',
    identifier='BBG000C05BD1',
    update_instrument_identifier_request=request)

# Print the response showing the updated ClientInternal ID
print('Instrument Name: ' + response.name + '\n', 
      'Figi: ' + response.identifiers['Figi'] + '\n',
      'ClientInternal: ' + response.identifiers['ClientInternal'] + '\n',
      'Isin: ' + response.identifiers['Isin']+ '\n',
      'Ticker: ' + response.identifiers['Ticker'] + '\n',
      'Lusid Instrument ID: ' + response.lusid_instrument_id + '\n')

### 8) Delete instruments

In addition to upserting instruments we can also delete them. When we delete an instrument it becomes unvailable for us to use from the time that we delete it onwards. If we want to look back in time at our instruments or transactions in those instruments we will be able to find them, they are not permanently deleted from the system.

For further usage of the delete instrument API call refer to the [LUSID API Docs: Delete Instrument](https://docs.lusid.com/#operation/DeleteInstrument).


*Run the cell below to delete our instrument universe*

In [ ]:
for figi in instruments.loc[:, 'figi'].values:

    response = api_factory.build(lusid.api.InstrumentsApi).delete_instrument(
        identifier_type='Figi',
        identifier=figi)
    
    print ('Instrument with FIGI {} deleted'.format(figi))